<div style="border-radius: 30px 0 30px 0px; border: 2px solid rgb(23, 35, 207); padding: 20px; background-color:rgb(0, 0, 0); text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0); width: 60%; margin: auto;">
    <h1 style="color:rgb(23, 35, 207); text-shadow: 2px 2px 4px rgba(0, 0, 0); font-weight: bold; margin-bottom: 10px; font-size: 36px;">Importing The Libraries
</h1>
</div>

In [1]:
from PIL import Image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')

In [28]:
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

zsh:1: command not found: kaggle


In [3]:
# import zipfile
# zip_ref = zipfile.ZipFile('chest-xray-pneumonia.zip', 'r')
# zip_ref.extractall('chest-xray-pneumonia')
# zip_ref.close()

<div style="border-radius: 30px 0 30px 0px; border: 2px solid rgb(23, 35, 207); padding: 20px; background-color:rgb(0, 0, 0); text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0); width: 60%; margin: auto;">
    <h1 style="color:rgb(23, 35, 207); text-shadow: 2px 2px 4px rgba(0, 0, 0); font-weight: bold; margin-bottom: 10px; font-size: 36px;">Move All Data To One Folder
</h1>
</div>

In [4]:
train_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/train"
val_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/val"
test_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/test"
all_data_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data"

os.makedirs(all_data_dir, exist_ok=True)

original_dirs = [train_dir, val_dir, test_dir]

for folder in original_dirs:
    for label in ['NORMAL', 'PNEUMONIA']:
        src_dir = os.path.join(folder, label)
        dest_dir = os.path.join(all_data_dir, label)
        os.makedirs(dest_dir, exist_ok=True)
        for file in os.listdir(src_dir):
            shutil.copy(os.path.join(src_dir, file), os.path.join(dest_dir, file))

print("All data has been consolidated into:", all_data_dir)

All data has been consolidated into: /Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data


<div style="border-radius: 30px 0 30px 0px; border: 2px solid rgb(23, 35, 207); padding: 20px; background-color:rgb(0, 0, 0); text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0); width: 60%; margin: auto;">
    <h1 style="color:rgb(23, 35, 207); text-shadow: 2px 2px 4px rgba(0, 0, 0); font-weight: bold; margin-bottom: 10px; font-size: 36px;">Count Imbalance Classes
</h1>
</div>

In [5]:
def count_images(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts

In [6]:
class_distribution = count_images(all_data_dir)

for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} images")

PNEUMONIA: 4274 images
NORMAL: 1584 images


### **NORMAL Class images need to be augmented**

In [7]:
normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/train"
augmented_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/augmented_normal"
os.makedirs(augmented_dir, exist_ok=True)

### **Augment NORMAL Images and Generate New Augmented Images**

In [8]:
normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/NORMAL"
augmented_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/augmented_normal"
os.makedirs(augmented_dir, exist_ok=True)

allowed_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

for filename in os.listdir(normal_dir)
    if not any(filename.lower().endswith(ext) for ext in allowed_extensions):
        continue

    img_path = os.path.join(normal_dir, filename)

    try:
        img = Image.open(img_path).resize((256, 256))
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        if img_array.ndim == 3:
            img_array = np.expand_dims(img_array, axis=-1)

        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=augmented_dir, save_prefix="aug", save_format="jpeg"):
            i += 1
            if i >= 5:
                break

    except Exception as e:
        print(f"Error processing file {filename}: {e}")

print(f"Augmented images saved to {augmented_dir}")

Augmented images saved to /Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/augmented_normal


### **Generate Total of "5448" new Augmented Images**

In [9]:
augmented_normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/augmented_normal"

augmented_images = os.listdir(augmented_normal_dir)
num_augmented_images = len(augmented_images)

print(f"Number of augmented images created: {num_augmented_images}")

Number of augmented images created: 5448


### **Move all New Augmented Images and Not Augmented to Single Folder**

In [10]:
normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/NORMAL"

augmented_images = os.listdir(augmented_normal_dir)

for img in augmented_images:
    shutil.copy(os.path.join(augmented_normal_dir, img), os.path.join(normal_dir, img))

print(f"Successfully copied {len(augmented_images)} augmented images to the NORMAL directory.")

Successfully copied 5448 augmented images to the NORMAL directory.


### **Total NORMAL Class Images become "7038"**

This makes again class imabalance issue
- Total NORMAL Class Images ==> "7038"
- Total NORMAL Class Images ==> "4274"

In [11]:
normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/NORMAL"

augmented_images = os.listdir(normal_dir)
num_augmented_images = len(augmented_images)

print(f"Number of augmented images created: {num_augmented_images}")

Number of augmented images created: 7032


### **Handle Class Imbalance Issue**
- Make same number of images in both classes to resolve class imbalance issue

In [12]:
import random

normal_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/NORMAL"
pneumonia_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/PNEUMONIA"
balanced_data_dir = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/balanced_data"

normal_images = os.listdir(normal_dir)
pneumonia_images = os.listdir(pneumonia_dir)

num_images_to_select = len(pneumonia_images)

selected_normal_images = random.sample(normal_images, num_images_to_select)

balanced_normal_dir = os.path.join(balanced_data_dir, "NORMAL")
balanced_pneumonia_dir = os.path.join(balanced_data_dir, "PNEUMONIA")

os.makedirs(balanced_normal_dir, exist_ok=True)
os.makedirs(balanced_pneumonia_dir, exist_ok=True)

for img in selected_normal_images:
    shutil.copy(os.path.join(normal_dir, img), balanced_normal_dir)

for img in pneumonia_images:
    shutil.copy(os.path.join(pneumonia_dir, img), balanced_pneumonia_dir)

print(f"Balanced data: {len(selected_normal_images)} NORMAL and {len(pneumonia_images)} PNEUMONIA images.")

Balanced data: 4274 NORMAL and 4274 PNEUMONIA images.


### **Issue Resolved**

In [13]:
normal_imgs = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/balanced_data/NORMAL"
pneumonia_imgs = "/Users/innovationinsight/Desktop/Umer/chest_xray/chest_xray/all_data/balanced_data/PNEUMONIA"

tol_nomral_images = os.listdir(normal_imgs)
num_nor_images = len(tol_nomral_images)

tol_pneu_images = os.listdir(pneumonia_imgs)
num_pneu_images = len(tol_pneu_images)

print(f"Number of images created: {num_nor_images}")
print(f"Number of images created: {num_pneu_images}")

Number of images created: 4274
Number of images created: 4274


#### **Sucess:**
- **Number of NORMAL class images == Number of PNEUMONIA Class Images**

### **For creating separate folder as:**
- **Train**
- **Test**
- **Validation**

### **Refer to <a href = "https://github.com/MuhammadUmerKhan/Medial-Pneumonia-Classification/blob/main/ipynb/Pneumonia_detection.ipynb">Pneumonia_detection.ipynb</a>**